In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingRegressor

# Task 1: Data Preparation

# Load the 5 data files into separate dataframes
df83 = pd.read_csv('83_Loeschcke_et_al_2000_Thorax_&_wing_traits_lab pops.csv')
df84 = pd.read_csv('84_Loeschcke_et_al_2000_Wing_traits_&_asymmetry_lab pops.csv')
df85 = pd.read_csv('85_Loeschcke_et_al_2000_Wing_asymmetry_lab_pops (1).csv')

plot_dir = 'plots'
os.makedirs(plot_dir, exist_ok=True)

for tdf in [df83, df84, df85]:
  for column in tdf.columns:
    if pd.api.types.is_numeric_dtype(tdf[column]):
     mean_value = tdf[column][tdf[column] != 0].mean()
     tdf[column] = tdf[column].replace(0, mean_value)
     tdf[column].fillna(mean_value, inplace=True)

df83.info()
df84.info()
df85.info()




In [ ]:
# Preprocess the 'Species' column to have a consistent format
df83['Species'] = df83['Species'].str.replace('D._', 'D.', regex=True)
df85['Species'] = df85['Species'].str.replace('D._', 'D.', regex=True)

common_cols = ['Species', 'Population', 'Latitude', 'Longitude', 'Year_start', 'Year_end', 'Temperature', 'Vial', 'Replicate', 'Sex']

# Merge the DataFrames based on common columns
merged_df = pd.merge(df83, df84[common_cols + ['Wing_area', 'Wing_shape', 'Wing_vein', 'Asymmetry_wing_area', 'Asymmetry_wing_shape', 'Asymmetry_wing_vein']], on=common_cols, how='left')

merged_df = pd.merge(merged_df, df85[common_cols + ['Asymmetry_l2', 'Asymmetry_l3p', 'Asymmetry_l3d', 'Asymmetry_lpd', 'Asymmetry_l3', 'Asymmetry_w1', 'Asymmetry_w2', 'Asymmetry_w3']], on=common_cols, how='left')

for column in merged_df.columns:
    if pd.api.types.is_numeric_dtype(merged_df[column]):
     mean_value = merged_df[column][merged_df[column] != 0].mean()
     merged_df[column] = merged_df[column].replace(0, mean_value)
     merged_df[column].fillna(mean_value, inplace=True)

merged_df.info()

# Display the merged DataFrame
print(merged_df)

current_dir = os.getcwd()

# Construct the file path for the merged data CSV file
merged_data_file = os.path.join(current_dir, 'merged_data.csv')

# Save the merged DataFrame to the CSV file, replacing if it already exists
merged_df.to_csv(merged_data_file, index=False)

In [ ]:
# Feature Selection and Data Preparation
columns_for_eda = ['Latitude', 'Longitude','Temperature', 'Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3', 'w1', 'w2', 'w3', 'wing_loading', 'Wing_area', 'Asymmetry_wing_area', 'Asymmetry_wing_vein', 'Asymmetry_l3p', 'Asymmetry_lpd', 'Asymmetry_l3']

for col in merged_df[columns_for_eda].columns:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

def analyze_correlation_heatmap(corr_matrix, threshold=0.7):
    analysis = ""
    columns = corr_matrix.columns

    for i in range(len(columns)):
        for j in range(i+1, len(columns)):
            col1 = columns[i]
            col2 = columns[j]
            corr_value = corr_matrix.iloc[i, j]

            if threshold <= corr_value < 1:
                description = "strong positive correlation"
                analysis += f"There is a {description} ({corr_value:.2f}) between '{col1}' and '{col2}'.\n"

    if not analysis:
        analysis = f"No strong correlations (above {threshold}) were found in the data."

    return analysis

# Generate correlation heatmap
correlation_matrix = merged_df[columns_for_eda].corr()
plt.figure(figsize=(18, 15))
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.yticks(rotation=0)
plt.title('Correlation Heatmap', fontsize=30)
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=1024)
plt.show()

analysis = analyze_correlation_heatmap(correlation_matrix, threshold=0.7)
print(analysis)

# Plot histogram for selected columns
histogram_columns = ['Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3', 'w1', 'w2', 'w3', 'wing_loading', 'Wing_area']
for col in histogram_columns:
    fig, ax = plt.subplots(figsize=(6, 4))
    merged_df[col].hist(ax=ax, bins=20)
    ax.set_title(f'Histogram of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Count')
    plt.tight_layout()
    hist_file = os.path.join(plot_dir, f'histogram_{col}.png')
    plt.savefig(hist_file, dpi=300, bbox_inches='tight')
    plt.close()
    plt.show()


In [ ]:
sex_counts = merged_df['Sex'].value_counts()
fig, ax = plt.subplots(figsize=(6, 4))
sex_counts.plot(kind='bar', ax=ax, rot=0)
ax.set_title('Distribution of Sex')
ax.set_xlabel('Sex')
ax.set_ylabel('Count')
plt.tight_layout()
bar_plot_file = os.path.join(plot_dir, 'sex_bar_plot.png')
plt.savefig(bar_plot_file, dpi=300, bbox_inches='tight')
plt.close()

In [ ]:
# Bar plot
plt.figure(figsize=(12, 8))
sns.countplot(data=merged_df, x='Population')
plt.title('Bar Plot of Population')
plt.xticks(rotation=45)
plt.show()

# Density plot
plt.figure(figsize=(12, 8))
sns.kdeplot(data=merged_df['Temperature'], fill=True)
plt.title('Density Plot of Temperature')
plt.xlabel('Temperature')
plt.ylabel('Density')
plt.show()

# Scatter plot matrix
columns = ['Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3', 'w1', 'w2', 'w3', 'wing_loading', 'Wing_area']
selected_df = merged_df[columns]

scatter_matrix = pd.plotting.scatter_matrix(selected_df, figsize=(15, 15))
plt.suptitle('Scatter Plot Matrix of Variables', x=0.5, y=0.92)
plt.show()

In [ ]:
# Prepare data for modeling
target_column = ['Thorax_length', 'l2', 'l3p', 'l3d', 'lpd', 'l3', 'w1', 'w2', 'w3', 'wing_loading', 'Wing_area', 'Asymmetry_wing_area', 'Asymmetry_wing_vein', 'Asymmetry_l3p', 'Asymmetry_lpd', 'Asymmetry_l3']

target = ['Thorax_length', 'l2', 'lpd', 'l3', 'w1', 'w2', 'w3', 'wing_loading', 'Wing_area']

df = merged_df[target_column]

# Convert columns to numeric, coercing errors to NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Drop rows with NaNs in any target column
df.dropna(subset=target, inplace=True)

y = df[target]
X = df


# Split data into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=250)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=250)



In [ ]:
models = {
    "KNN": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "SVR": MultiOutputRegressor(SVR()),
    "Linear Regression": LinearRegression(),
    "MLP": MLPRegressor(max_iter=5000)
}

# Define hyperparameters for grid search
params = {
    "KNN": {'n_neighbors': range(1, 21)},
    "Decision Tree": {'max_depth': range(1, 21)},
    "Random Forest": {'n_estimators': [10, 50, 100, 200], 'max_depth': range(1, 21)},
    "SVR": {'estimator__C': [0.1, 1, 10], 'estimator__kernel': ['linear', 'rbf']},
    "Linear Regression": {},
    "MLP": {'hidden_layer_sizes': [(10,), (50,), (100,)], 'alpha': [0.0001, 0.001, 0.01]}
}

# Train and evaluate models
results = {}
for name, model in models.items():
    grid_search = GridSearchCV(model, params[name], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred_train = best_model.predict(X_train)
    y_pred_val = best_model.predict(X_val)
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_train = r2_score(y_train, y_pred_train)
    r2_val = r2_score(y_val, y_pred_val)
    results[name] = {"best_model": best_model, "mse_train": mse_train, "mse_val": mse_val, "r2_train": r2_train, "r2_val": r2_val}

In [ ]:
# Compare performance
for name, result in results.items():
    print(f"{name}: MSE (Train) = {result['mse_train']}, MSE (Val) = {result['mse_val']}, R^2 (Train) = {result['r2_train']}, R^2 (Val) = {result['r2_val']}")

colors = ['blue', 'green', 'red', 'purple', 'orange', 'yellow']

plt.figure(figsize=(8, 6))
plt.bar(results.keys(), [result['mse_val'] for result in results.values()], color=colors)
plt.xlabel('Model')
plt.ylabel('Mean Squared Error')
plt.title('Model Performance Comparison')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('model_performance_comparison.png')
plt.show()

plt.figure(figsize=(8, 6))
plt.bar(results.keys(), [result['r2_val'] for result in results.values()], color=colors)
plt.xlabel('Model')
plt.ylabel('R² Value')
plt.title('Model Performance Comparison - R²')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('model_performance_comparison_r2.png')
plt.show()

In [ ]:
for name, result in results.items():
    model = result["best_model"]
    y_pred_test = model.predict(X_test)

    # Plotting for each target variable
    for target in y_test.columns:  # Iterate over column names
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.scatter(y_test[target], y_pred_test[:, y_test.columns.get_loc(target)], alpha=0.5)
        ax.set_xlabel(f'Actual {target}')
        ax.set_ylabel(f'Predicted {target}')
        ax.set_title(f'{name}: Actual vs Predicted {target}')
        fig.savefig(f'{name}_{target}_actual_vs_predicted.png', bbox_inches='tight')
        plt.show()
        plt.close(fig)

In [ ]:
# Define a wider range of hyperparameters for KNN
params_knn = {'n_neighbors': range(1, 51), 'weights': ['uniform', 'distance'], 'p': [1, 2]}

# Perform grid search with the extended parameter grid
grid_search_knn = GridSearchCV(KNeighborsRegressor(), params_knn, cv=5, scoring='neg_mean_squared_error')
grid_search_knn.fit(X_train, y_train)

# Get the best KNN model
knn_best_extended = grid_search_knn.best_estimator_

# Define a wider range of hyperparameters for MLP
params_mlp = {'hidden_layer_sizes': [(10,), (50,), (100,), (10, 10), (50, 50), (100, 100)], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}

# Perform grid search with the extended parameter grid
grid_search_mlp = GridSearchCV(MLPRegressor(max_iter=2000), params_mlp, cv=5, scoring='neg_mean_squared_error')
grid_search_mlp.fit(X_train, y_train)

# Get the best MLP model
mlp_best_extended = grid_search_mlp.best_estimator_

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X_train)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Define MSE values for existing models
existing_models_mse = {}

for name, result in results.items():
    existing_models_mse[name] = result['mse_val']

tuned_models_mse = {
    "KNN (Tuned)": abs(grid_search_knn.best_score_),
    "MLP (Tuned)": abs(grid_search_mlp.best_score_),
}


# Combine MSE values for both existing and tuned models
all_models_mse = {**existing_models_mse, **tuned_models_mse}

# Create a bar plot to compare MSE values for all models
plt.figure(figsize=(10, 6))
plt.bar(all_models_mse.keys(), all_models_mse.values(), color=['blue' if model.endswith('(Tuned)') else 'green' for model in all_models_mse.keys()])
plt.xlabel('Model')
plt.ylabel('Mean Squared Error')
plt.title('Model Performance Comparison Before and After Tuning')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Perform PCA
pca = PCA(n_components=5)
X_pca = pca.fit_transform(X_train)  # Fit PCA on training data

# Plot explained variance ratio
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance by Principal Components')
plt.show()

# Train models on PCA-transformed data
results_pca = {}
for name, model in models.items():
    grid_search = GridSearchCV(model, params[name], cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_pca, y_train)
    best_model = grid_search.best_estimator_

    y_pred_train = best_model.predict(X_pca)
    y_pred_val = best_model.predict(pca.transform(X_val))  # Transform validation data using the same PCA

    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_train = r2_score(y_train, y_pred_train)
    r2_val = r2_score(y_val, y_pred_val)

    results_pca[name] = {"best_model": best_model, "mse_train": mse_train, "mse_val": mse_val, "r2_train": r2_train, "r2_val": r2_val}

# Print results with PCA
for name, result in results_pca.items():
    print(f"{name} Results (with PCA):")
    print(f"  MSE (Train): {result['mse_train']:.4f}")
    print(f"  MSE (Validation): {result['mse_val']:.4f}")
    print(f"  R-squared (Train): {result['r2_train']:.4f}")
    print(f"  R-squared (Validation): {result['r2_val']:.4f}")
    print()

colors = ['blue', 'green', 'red', 'purple', 'orange', 'yellow']

plt.figure(figsize=(8, 6))
plt.bar(results_pca.keys(), [result['mse_val'] for result in results_pca.values()], color=colors)
plt.xlabel('Model')
plt.ylabel('Mean Squared Error')
plt.title('Model Performance Comparison')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('model_performance_comparison.png')
plt.show()



In [ ]:

model_names = list(results_pca.keys())
# Extract MSE and R2 values for models with and without PCA
mse_val_pca = [result['mse_val'] for result in results_pca.values()]
r2_val_pca = [result['r2_val'] for result in results_pca.values()]
mse_val_no_pca = [results[model]['mse_val'] for model in model_names]
r2_val_no_pca = [results[model]['r2_val'] for model in model_names]

bar_width = 0.35
index = np.arange(len(model_names))

# Plotting MSE comparison
plt.figure(figsize=(12, 6))
plt.bar(index, mse_val_no_pca, bar_width, label='Without PCA', color=colors)
plt.bar(index + bar_width, mse_val_pca, bar_width, label='With PCA', color=[plt.cm.Paired(i/len(colors)) for i in range(len(colors))])
plt.xlabel('Model')
plt.ylabel('Mean Squared Error')
plt.title('Model Performance Comparison (MSE)')
plt.xticks(index + bar_width / 2, model_names, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('model_performance_comparison_mse.png')
plt.show()

# Plotting R2 comparison
plt.figure(figsize=(12, 6))
plt.bar(index, r2_val_no_pca, bar_width, label='Without PCA', color=colors)
plt.bar(index + bar_width, r2_val_pca, bar_width, label='With PCA', color=[plt.cm.Paired(i/len(colors)) for i in range(len(colors))])
plt.xlabel('Model')
plt.ylabel('R² Score')
plt.title('Model Performance Comparison (R²)')
plt.xticks(index + bar_width / 2, model_names, rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('model_performance_comparison_r2.png')
plt.show()

In [ ]:
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown']

# Extract results from results_pca dictionary
mse_train = [results_pca[name]['mse_train'] for name in models]
mse_val = [results_pca[name]['mse_val'] for name in models]
r2_train = [results_pca[name]['r2_train'] for name in models]
r2_val = [results_pca[name]['r2_val'] for name in models]

# Plot MSE
plt.figure(figsize=(10, 6))
bar_width = 0.35
index = np.arange(len(models))
plt.bar(index, mse_train, bar_width, color=[colors[i] for i in range(len(models))], alpha=0.5, label='Train')
plt.bar(index + bar_width, mse_val, bar_width, color=[colors[i] for i in range(len(models))], alpha=0.8, label='Validation')
plt.xticks(index + bar_width / 2, models, rotation=45, ha='right')
plt.ylabel('Mean Squared Error')
plt.title('Mean Squared Error (Train vs Validation)')
plt.legend()
plt.tight_layout()
plt.savefig('plots/mse_plot.png')
plt.show()

# Plot R-squared
plt.figure(figsize=(10, 6))
plt.bar(index, r2_train, bar_width, color=[colors[i] for i in range(len(models))], alpha=0.5, label='Train')
plt.bar(index + bar_width, r2_val, bar_width, color=[colors[i] for i in range(len(models))], alpha=0.8, label='Validation')
plt.xticks(index + bar_width / 2, models, rotation=45, ha='right')
plt.ylabel('R-squared')
plt.title('R-squared (Train vs Validation)')
plt.legend()
plt.tight_layout()
plt.savefig('plots/r2_plot.png')
plt.show()